In [12]:
!pip3 install -U selenium
!pip3 install webdriver-manager

In [13]:
# Chrome -> chromedriver
# firefox -> geckodriver

In [14]:
# passo 1: pegar a cotacao dolar
# passo 2: pegar a cotacao euro
# passo 3: pegar a cotacao ouro
# passo 4: atualizar a base de dados
# passo 5: recalcular os precos
# passo 6: exportar a base de dados

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# passo 1: pegar a cotacao dolar
# abrir o navegador
driver = webdriver.Chrome()

# abrir o google
driver.get('https://www.google.com.br/')

# pesquisar a cotacao do dolar
driver.find_element('xpath', 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')

driver.find_element('xpath', 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

dolar_value = driver.find_element('xpath',
                                 '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(dolar_value)


5.1728


In [16]:
# abrir o google
driver.get('https://www.google.com.br/')

# passo 2: pegar a cotacao euro
driver.find_element('xpath', 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')

driver.find_element('xpath', 
                    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

euro_value = driver.find_element('xpath',
                                 '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


print(euro_value)

5.2900673760000005


In [17]:
# abrir o site com cotação do ouro
driver.get('https://www.melhorcambio.com/ouro-hoje')

# passo 3: pegar a cotacao ouro
gold_value = driver.find_element('xpath', 
                    '//*[@id="comercial"]').get_attribute('value')

print(gold_value)

293,37


In [18]:
# transformar para o formato de concatenação do python
gold_value = gold_value.replace(',','.')
print(gold_value)

# Fechar o navegador 
driver.quit()

293.37


In [19]:
import pandas as pd

# passo 4: atualizar a base de dados

# importar a base de dados
dataframe = pd.read_excel('data/Produtos.xlsx')

display(dataframe)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [20]:
# passo 5: recalcular os precos

# atualizar as cotações 
dataframe.loc[dataframe['Moeda'] == 'Dólar', 'Cotação'] = float(dolar_value)
dataframe.loc[dataframe['Moeda'] == 'Euro', 'Cotação'] = float(euro_value)
dataframe.loc[dataframe['Moeda'] == 'Ouro', 'Cotação'] = float(gold_value)

display(dataframe)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.172800,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.290067,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.172800,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.172800,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.290067,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.172800,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,293.370000,7000.00,1.15,8050.000


In [21]:
# atulizar o Preço de Compra = Preço Original * Cotação
dataframe['Preço de Compra'] = dataframe['Preço Original'] * dataframe['Cotação']

# atulizar o Preço de Venda = Preço de Compra * Margem
dataframe['Preço de Venda'] = dataframe['Preço de Compra'] * dataframe['Margem']

display(dataframe)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.172800,5172.748272,1.40,7241.847581
1,Carro Renault,4500.00,Euro,5.290067,23805.303192,2.00,47610.606384
2,Notebook Dell,899.99,Dólar,5.172800,4655.468272,1.70,7914.296062
3,IPhone,799.00,Dólar,5.172800,4133.067200,1.70,7026.214240
4,Carro Fiat,3000.00,Euro,5.290067,15870.202128,1.90,30153.384043
5,Celular Xiaomi,480.48,Dólar,5.172800,2485.426944,2.00,4970.853888
6,Joia 20g,20.00,Ouro,293.370000,5867.400000,1.15,6747.510000


In [22]:
# passo 6: exportar a base de dados

# index=False remove a coluna com o índice de linha
dataframe.to_excel('Produtos-novo.xlsx', index=False)